In [1]:
import pandas as pd
import plotly.express as px
import json
from glob import glob


from data_processing import all_tokens


def get_state_of_destination(json_path: str) -> pd.DataFrame:
    with open(json_path, "r") as fin:
        data = json.load(fin)
    state_of_destinations = data["sod"]["destStates"]
    raw_df = pd.DataFrame.from_records(state_of_destinations)
    raw_df["Date"] = pd.to_datetime(data["sod"]["currentTimestamp"], unit="s")
    return raw_df


paths = glob("../data/*")
raw_df = pd.concat([get_state_of_destination(p) for p in paths])

In [2]:
raw_df.columns

Index(['snapshotTimestamp', 'name', 'address', 'poolType', 'pool',
       'underlying', 'underlyingTokens', 'underlyingReserves', 'totalAprIn',
       'totalAprOut', 'incentiveAPR', 'destLPValue',
       'discountViolationAddFlag', 'discountViolationTrim1Flag',
       'discountViolationTrim2Flag', 'ownedShares', 'totSupply', 'safePrice',
       'spotPrice', 'tokenSpotPrice', 'tokenSafePrice', 'Date', 'flipFlag'],
      dtype='object')

In [3]:
def build_lp_composition_over_time_df(raw_df: pd.DataFrame) -> pd.DataFrame:
    lp_composition_over_time_df = raw_df[["name", "Date"]].copy()
    lp_composition_over_time_df["eth_value_in_destination"] = raw_df.apply(
        _add_eth_value_in_destination, axis=1
    )
    return lp_composition_over_time_df


def _add_eth_value_in_destination(row: dict):
    return (int(row["ownedShares"]) / 1e18) * row["safePrice"]


def make_autopool_LP_composition_over_time_chart(raw_df: list[pd.DataFrame]):
    lp_composition_over_time_df = build_lp_composition_over_time_df(raw_df)
    eth_in_destination_df = lp_composition_over_time_df.pivot_table(
        columns="name", index="Date", values="eth_value_in_destination"
    ).sort_index()
    # only look at destinatios we have touched,
    # eg where at least one of the values in eth_value_in_destination is not 0
    # this is just to make the legend cleaner
    eth_in_destination_df = eth_in_destination_df.loc[
        :, (eth_in_destination_df != 0).any(axis=0)
    ]
    fig = px.bar(
        eth_in_destination_df.resample("1D").ffill(),
        title="LP Composition Over Time In ETH",
        labels={"value": "ETH"},
    )
    return fig


lp_composition_over_time_fig = make_autopool_LP_composition_over_time_chart(raw_df) # These are the weights
lp_composition_over_time_fig.show()

Does this make sense

In [15]:
def build_asset_composition_over_time_df(raw_df: pd.DataFrame) -> pd.DataFrame:
    name_time_df = raw_df[["name", "Date"]].copy()
    
    date_assets_owned_records = raw_df.apply(
        _add_eth_value_in_each_asset, axis=1
    )
    assets_owned_df = pd.DataFrame.from_records(date_assets_owned_records)
    
    return pd.concat([name_time_df, assets_owned_df], axis=0) 


def _add_eth_value_in_each_asset(row: dict):
    shares_owned = int(row["ownedShares"]) / 1e18
    # this will work if we have total supply, can be backed out using destination eth value but
    # using total supply is clearer
    portion_of_pool_owned = shares_owned / (int(row["totSupply"]) / 1e18)
    portion_of_tokens_owned = {'Date': row['Date']}

    for token, underlyingReserve, tokenSafePrice in zip(row["underlyingTokens"], row["underlyingReserves"], row["tokenSafePrice"]):
        portion_of_tokens_owned[token] = portion_of_pool_owned * (int(underlyingReserve) / 1e18) * tokenSafePrice

    return portion_of_tokens_owned # dict[token: eth value of this token ]

a = build_asset_composition_over_time_df(raw_df)

ZeroDivisionError: float division by zero